In [1]:
] activate

 Activating environment at `/tank/home/ciaran/DCSideBatteryModeling/Project.toml`


In [4]:
using LinearAlgebra
using OrdinaryDiffEq #Gets the solvers
using Plots
plotly()
using PowerSystems
using DelimitedFiles

include(joinpath(pwd(), "DCSideBatteryModeling", "DCSideBatteryModeling.jl"))

┌ Info: Precompiling OrdinaryDiffEq [1dea7af3-3e70-54e6-95c3-0bf5283fa5ed]
└ @ Base loading.jl:1260
┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1260
┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots /home/ciaran/.julia/packages/Plots/ViMfq/src/backends.jl:373
┌ Info: Precompiling PowerSystems [bcd98974-b02a-5e2f-9ee0-a103f5c450dd]
└ @ Base loading.jl:1260
┌ Info: Precompiling ModelingToolkit [961ee093-0014-501f-94e3-6117800e7a78]
└ @ Base loading.jl:1260


In [5]:
include(joinpath(pwd(), "data","make_data.jl"))

┌ Info: The PSS(R)E parser currently supports buses, loads, shunts, generators, branches, transformers, and dc lines
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/cvPQ9/src/parsers/pm_io/common.jl:26
┌ Info: Constructing System from Power Models
│   data["name"] = omib
│   data["source_type"] = pti
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/cvPQ9/src/parsers/power_models_data.jl:39
┌ Info: Created InMemoryTimeSeriesStorage
└ @ InfrastructureSystems /home/ciaran/.julia/packages/InfrastructureSystems/2F6Pq/src/in_memory_time_series_storage.jl:24
┌ Info: Reading bus data
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/cvPQ9/src/parsers/power_models_data.jl:140
┌ Info: Reading generator data
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/cvPQ9/src/parsers/power_models_data.jl:418
┌ Info: Reading branch data
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/cvPQ9/src/parsers/power_models_data.jl:575
┌ Info: Reading branch data
└ @

In [6]:
omib_sys = System(joinpath(pwd(), "data", "OMIB_inverterDCside.json"))

┌ Info: Created InMemoryTimeSeriesStorage
└ @ InfrastructureSystems /home/ciaran/.julia/packages/InfrastructureSystems/2F6Pq/src/in_memory_time_series_storage.jl:24
┌ Warning: There are no Generator Components in the System
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/cvPQ9/src/utils/IO/system_checks.jl:56
┌ Warning: System peak load (50.0) exceeds total capacity capability (0).
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/cvPQ9/src/utils/IO/system_checks.jl:72


,ConcreteType,SuperTypes
,String,String
1,Area,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any
2,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any
3,"DynamicInverter{AverageConverter,OuterControl{VirtualInertia,ReactivePowerDroop},CurrentControl,FixedDCSource,KauraPLL,LCLFilter}",DynamicInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any
4,GenericBattery,Storage <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any
5,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any
6,LoadZone,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any
7,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any


## Experiment Parameters

In [144]:
InitialLoad = 0.2 # Initial value of the active power demand
LoadStep = 0.5 # Active power step change
CapSize = 2000e-6 # DC-link capacitor size
StartTime=0 # Start time of time-domain simulation 
EndTime=0.05 # End time of time-domain simulation
TimeStep= 0.0001 # Timestep of saved time series

cap_range = 2000e-6:500e-6:7000e-6 # DC-link capacitor sizing
kpred_range = 0:0.25:5 # Range of predictive gain

0.0:0.25:5.0

In [148]:
exp_params = ["Cb_dc", "cdc", "pl", "kpvb", "kivb", "kpib", "kiib", "kpred", "vdcʳ"] #This is a list of parameters that we will adjust during experiment 

9-element Array{String,1}:
 "Cb_dc"
 "cdc"
 "pl"
 "kpvb"
 "kivb"
 "kpib"
 "kiib"
 "kpred"
 "vdcʳ"

## Instantiate ODE Models

In [150]:
parameter_mapping = instantiate_parameters(omib_sys)
M_4th= instantiate_4th_order_model(omib_sys)
M_2nd= instantiate_2nd_order_model(omib_sys)
M_0th= instantiate_0th_order_model(omib_sys)

ModelOperatingPoint


In [86]:
_parameter_names = [string(x.first) for x in parameter_mapping]; # Extract list of parameter values
_parameter_values = [x.second for x in parameter_mapping]; # Extract list of parameter values

In [149]:
params_indx = Dict{String,Integer}()
for p in exp_params
    params_indx[p] = findfirst(isequal(p), _parameter_names) # Dict mapping exp_parms to index corresponfing index in model
end

In [95]:
_parameter_values[params_indx["cdc"]]=CapSize/_parameter_values[params_indx["Cb_dc"]]
_parameter_values[params_indx["pl"]]=InitialLoad

0.2

## Make copy of parameters and change value of load at t=0

In [96]:
disturbance=deepcopy(_parameter_values)
disturbance[params_indx["pl"]]=InitialLoad+LoadStep # Load step change from 0.2 p.u. to 0.7 p.u.

0.7

## Instantiate and simualte 4$^{th}$-, 2$^{nd}$- and 0$^{th}$-order model for non-optimized gains

In [97]:
sim_4th_nonopt = instantiate_4th_order_dae(disturbance, M_4th; tspan = (StartTime, EndTime))
sol_4th_nonopt = solve(sim_4th_nonopt, Rodas5(), saveat = TimeStep);

In [98]:
sim_2nd_nonopt = instantiate_2nd_order_dae(disturbance, M_2nd; tspan = (StartTime, EndTime))
sol_2nd_nonopt = solve(sim_2nd_nonopt, Rodas5(), saveat = TimeStep);

In [99]:
sim_0th_nonopt = instantiate_0th_order_dae(disturbance, M_0th; tspan = (StartTime, EndTime))
sol_0th_nonopt = solve(sim_0th_nonopt, Rodas5(), saveat = TimeStep);

## Plotting DC-voltage for all three models

In [100]:
plot(sol_4th_nonopt, vars=(0,13), label="4th-order", ylabel="DC-link Voltage (p.u.)")
plot!(sol_2nd_nonopt, vars=(0,13), label="2nd-order")
plot!(sol_0th_nonopt, vars=(0,13), label="0th-order")

<!DOCTYPE html>
 
 
 Plots.jl

## Save results to array and write to .csv file

In [101]:
untuned_4th = [sol_4th_nonopt.t transpose(Array(sol_4th_nonopt))]
untuned_2nd = [sol_2nd_nonopt.t transpose(Array(sol_2nd_nonopt))]
untuned_0th = [sol_0th_nonopt.t transpose(Array(sol_0th_nonopt))];

In [113]:
fileName=string("results/dae_sim/untuned_4th_", Int(floor(CapSize/1e-6)), "uF.csv")
writedlm( fileName,  untuned_4th, ',')
fileName=string("results/dae_sim/untuned_2nd_", Int(floor(CapSize/1e-6)), "uF.csv")
writedlm( fileName,  untuned_2nd, ',')
fileName=string("results/dae_sim/untuned_0th_", Int(floor(CapSize/1e-6)), "uF.csv")
writedlm( fileName,  untuned_0th, ',')

## Load optimal gains from parameter_search.ipynb

In [114]:
fileName=string("results/stable_gains/capSize_", Int(floor(CapSize/1e-6)), "uF.csv") 
gains = readdlm(fileName, ',') # Loading set of stable gains 
fileName=string("results/optimal_gains/capSize_", Int(floor(CapSize/1e-6)), "uF.csv")
optIndex = readdlm( fileName, ',') # Loading index of optimal set of gains

2×1 Array{Float64,2}:
 4443.0
    0.06802762588059204

## Set controller gains to optimal values

In [115]:
_parameter_values[params_indx["kpvb"]]=gains[Int(optIndex[1]),1] #K_p^v
_parameter_values[params_indx["kivb"]]=gains[Int(optIndex[1]),2] #K_i^v
_parameter_values[params_indx["kpib"]]=gains[Int(optIndex[1]),3] #K_p^i
_parameter_values[params_indx["kiib"]]=gains[Int(optIndex[1]),4] #K_i^i

8.0

## Re-calculate initial conditons

In [116]:
M_4th(_parameter_values)
M_2nd(_parameter_values)
M_0th(_parameter_values)

21-element Array{Float64,1}:
  1.0067935237422023
 -0.040061818677185806
  0.2032828250035323
  0.058506521912278175
  0.2003183879749555
 -0.01599274198690052
  0.2023199540292933
  0.008076270122532618
  0.0002721717227920591
 -2.1729269003940933e-5
  0.07044778826693796
 -0.002789251686115313
  1.084409521544636
  0.4294351687452164
 -7.084994987786322e-19
  0.0706567181744013
  2.5036080038079005e-38
  4.600046756145387e-9
 -4.3790577010150533e-47
  0.0
  1.4375146112954338e-13

## Make copy of parameters and change value of load at t=0

In [117]:
disturbance=deepcopy(_parameter_values)
disturbance[params_indx["pl"]]=InitialLoad+LoadStep # Load step change from 0.2 p.u. to 0.7 p.u.

0.7

## Instantiate and simulate 4$^{th}$-, 2$^{nd}$- and 0$^{th}$-order model with optimized gains

In [118]:
sim_4th_opt = instantiate_4th_order_dae(disturbance, M_4th; tspan = (StartTime, EndTime))
sol_4th_opt = solve(sim_4th_opt, Rodas5(), saveat = TimeStep);

In [119]:
sim_2nd_opt = instantiate_2nd_order_dae(disturbance, M_2nd; tspan = (StartTime, EndTime))
sol_2nd_opt = solve(sim_2nd_opt, Rodas5(), saveat = TimeStep);

In [120]:
sim_0th_opt = instantiate_0th_order_dae(disturbance, M_0th; tspan = (StartTime, EndTime))
sol_0th_opt = solve(sim_0th_opt, Rodas5(), saveat = TimeStep);

## Plotting DC-voltage for all three models

In [121]:
plot(sol_4th_opt, vars=(0,13), label="4th-order", ylabel="DC-link Voltage (p.u.)")
plot!(sol_2nd_opt, vars=(0,13), label="2nd-order")
plot!(sol_0th_opt, vars=(0,13), label="0th-order")

<!DOCTYPE html>
 
 
 Plots.jl

## Save results to array and write to .csv file

In [122]:
tuned_4th = [sol_4th_opt.t transpose(Array(sol_4th_opt))]
tuned_2nd = [sol_2nd_opt.t transpose(Array(sol_2nd_opt))]
tuned_0th = [sol_0th_opt.t transpose(Array(sol_0th_opt))];

In [123]:
fileName=string("results/dae_sim/tuned_4th_", Int(floor(CapSize/1e-6)), "uF.csv")
writedlm( fileName,  tuned_4th, ',')
fileName=string("results/dae_sim/tuned_2nd_", Int(floor(CapSize/1e-6)), "uF.csv")
writedlm( fileName,  tuned_2nd, ',')
fileName=string("results/dae_sim/tuned_0th_", Int(floor(CapSize/1e-6)), "uF.csv")
writedlm( fileName,  tuned_0th, ',')

# Performance imporvement from increasing predictive gain

In [124]:
_parameter_values[params_indx["kpred"]]=0 #K_pred
M_0th(_parameter_values) # Recalcualte initial conditons 

disturbance=deepcopy(_parameter_values)
disturbance[params_indx["pl"]]=InitialLoad+LoadStep # Load step change from 0.2 p.u. to 0.7 p.u.

sim_k0 = instantiate_0th_order_dae(disturbance, M_0th; tspan = (StartTime, EndTime))
sol_k0 = solve(sim_k0, Rodas5(), saveat = TimeStep);

In [125]:
_parameter_values[params_indx["kpred"]]=1 #K_pred
M_0th(_parameter_values) # Recalcualte initial conditons 

disturbance=deepcopy(_parameter_values)
disturbance[params_indx["pl"]]=InitialLoad+LoadStep # Load step change from 0.2 p.u. to 0.7 p.u.

sim_k1 = instantiate_0th_order_dae(disturbance, M_0th; tspan = (StartTime, EndTime))
sol_k1 = solve(sim_k1, Rodas5(), saveat = TimeStep);

In [126]:
_parameter_values[params_indx["kpred"]]=2 #K_pred
M_0th(_parameter_values) # Recalcualte initial conditons 

disturbance=deepcopy(_parameter_values)
disturbance[params_indx["pl"]]=InitialLoad+LoadStep # Load step change from 0.2 p.u. to 0.7 p.u.

sim_k2 = instantiate_0th_order_dae(disturbance, M_0th; tspan = (StartTime, EndTime))
sol_k2 = solve(sim_k2, Rodas5(), saveat = TimeStep);

In [127]:
_parameter_values[params_indx["kpred"]]=5 #K_pred
M_0th(_parameter_values) # Recalcualte initial conditons 

disturbance=deepcopy(_parameter_values)
disturbance[params_indx["pl"]]=InitialLoad+LoadStep # Load step change from 0.2 p.u. to 0.7 p.u.

sim_k5 = instantiate_0th_order_dae(disturbance, M_0th; tspan = (StartTime, EndTime))
sol_k5 = solve(sim_k5, Rodas5(), saveat = TimeStep);

## Plotting DC-voltage for various predictive gain

In [128]:
plot(sol_k0, vars=(0,13), xlims = (0,0.025), label="k_pred=0", ylabel="DC-link Voltage (p.u.)")
plot!(sol_k1, vars=(0,13), xlims = (0,0.025), label="k_pred=1")
plot!(sol_k2, vars=(0,13), xlims = (0,0.025), label="k_pred=2")
plot!(sol_k5, vars=(0,13), xlims = (0,0.025), label="k_pred=5")

<!DOCTYPE html>
 
 
 Plots.jl

## Save results to array and write to .csv file

In [129]:
kpred_0 = [sol_k0.t transpose(Array(sol_k0))]
kpred_1 = [sol_k1.t transpose(Array(sol_k1))]
kpred_2 = [sol_k2.t transpose(Array(sol_k2))];
kpred_5 = [sol_k5.t transpose(Array(sol_k5))];

In [130]:
writedlm( "results/dae_sim/kpred_0.csv",  kpred_0, ',')
writedlm( "results/dae_sim/kpred_1.csv",  kpred_1, ',')
writedlm( "results/dae_sim/kpred_2.csv",  kpred_2, ',')
writedlm( "results/dae_sim/kpred_5.csv",  kpred_5, ',')

## Calculating reduction in voltage nadir

In [145]:
nadir_reduction = 100*(minimum(kpred_5[:,14])-minimum(kpred_0[:,14]))/(_parameter_values[params_indx["vdcʳ"]]-minimum(kpred_0[:,14]))

19.395725180396084

## Sweeping DC_link Capacitor size and predictive gain to generate heatmap

In [147]:
heat_map_values=reshape([],0,3);
for cap in cap_range
    for k in kpred_range
        _parameter_values[params_indx["cdc"]]=cap/_parameter_values[params_indx["Cb_dc"]] # Adjusting DC-link capacitor size
        fileName=string("results/stable_gains/capSize_", Int(floor(cap/1e-6)), "uF.csv")
        gains = readdlm(fileName, ',', Float64) # Loading set of stable gains
        fileName=string("results/optimal_gains/capSize_", Int(floor(cap/1e-6)), "uF.csv")
        optIndex = readdlm(fileName, ',', Float64) # Loading index of optimal gain
        _parameter_values[params_indx["kpvb"]]=gains[Int(optIndex[1]),1] #K_p^v
        _parameter_values[params_indx["kivb"]]=gains[Int(optIndex[1]),2] #K_i^v
        _parameter_values[params_indx["kpib"]]=gains[Int(optIndex[1]),3] #K_p^i
        _parameter_values[params_indx["kiib"]]=gains[Int(optIndex[1]),4] #K_i^i
        _parameter_values[params_indx["kpred"]]=k #K_pred
        M_0th(_parameter_values) #Initializing 0th order model

        disturbance=deepcopy(_parameter_values)
        disturbance[params_indx["pl"]]=InitialLoad+LoadStep # Step change in load from 0.2 p.u. to 0.7 p.u.

        sim = instantiate_0th_order_dae(disturbance, M_0th; tspan = (StartTime, EndTime))
        sol = solve(sim, Rodas5(), saveat = TimeStep);
        solArray = Array(sol)
        nadir=100*maximum(_parameter_values[params_indx["vdcʳ"]].-solArray[13,:]) # Calculating nadir drop in % of Base Votlage 
        
        heat_map_values = [heat_map_values; [floor(cap/1e-3) k round(nadir, digits=3)]] # Concatenating results
    end
end
writedlm( "results/dae_sim/heat_map_matrix.csv",  heat_map_values, ',') # Writing array to .csv file